In [ ]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint



# Creating the lists we want to write into
titles = []
years = []
time = []
video=[]
imdb_ratings = []
metascores = []
votes = []
us_gross = []
genres=[]
genre1=[]
genre2=[]
genre3=[]
plot=[]
director=[]
stars=[]
primary_genre=[]



# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}


pages = np.arange(1, 2000, 50)
pages


genres_list=['adventure','fantasy','comedy,romance','action,comedy','thriller','mystery','crime','animation','comedy','sci-fi','horror','romance','action']
#genres_list=['adventure']





for gen in genres_list:
    for pg in pages:
          # Getting the contents from the each url
          link='https://www.imdb.com/search/title/?genres='+str(gen)+'&start='+str(pg)+'&explore=title_type,genres&ref_=adv_nxt'
          page = requests.get(link, headers=headers)
          soup = BeautifulSoup(page.text, 'html.parser')
          # Aiming the part of the html we want to get the information from
          movie_div = soup.find_all('div', class_='lister-item mode-advanced')
          
          # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
          # Waiting time between requests for a number between 2-10 seconds
          for container in movie_div:
              # Scraping the movie's name
              name = container.h3.a.text
              titles.append(name)

              primary_genre.append(str(gen))
              
              # Scraping the movie's year
              year = container.h3.find('span', class_='lister-item-year').text
              years.append(year)
              
              # Scraping the movie's length
              runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
              time.append(runtime)
              
              # Scraping the Video Type
              video_type = container.find('span', class_='certificate').text if container.p.find('span', class_='certificate') else '-'
              video.append(video_type)
      
              # Scraping the rating
              imdb = container.strong.text if container.strong else '-'
              imdb_ratings.append(imdb)
              
              # Scraping the metascore
              m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
              metascores.append(m_score)
              
              # Scraping votes and gross earnings
              nv = container.find_all('span', attrs={'name':'nv'})
              vote = nv[0].text if len(nv) > 1 else '-'
              votes.append(vote)
              grosses = nv[1].text if len(nv) > 1 else '-'
              us_gross.append(grosses)
    
              # Scraping genre and split
              genre = container.find('span', class_='genre').text if container.find('span', class_='genre') else '-'
              genre_list=genre.split(',')
              genres.append(genre[1:])
              if len(genre_list)==3:
               first=genre_list[0][1:] or '-'
               second=genre_list[1] or '-'
               third=genre_list[2] or '-'
               genre1.append(first)
               genre2.append(second)
               genre3.append(third)
              elif len(genre_list)==2:
               first=genre_list[0][1:] or '-'
               second=genre_list[1] or '-'
               genre1.append(first)
               genre2.append(second)
               genre3.append('-')
              elif len(genre_list)==1:
               first=genre_list[0][1:] or '-'
               genre1.append(first)
               genre2.append('-')
               genre3.append('-')
              
              # Scraping movie plot
              movie_plot = container.findAll("p", {"class": "text-muted"})
              #print()
              movie_plot=movie_plot[1].text
              movie_plot=movie_plot[movie_plot.find("\n")+1:]
              plot.append(movie_plot)
    
              #scraping director name 
              name=container.select("p:nth-of-type(3)")[0].text.split('|')
              if len(name)==2:
                  director_name=name[0].strip()
                  actor_name=name[1].strip()
                  if director_name.find("Director")!=-1:
                   director_name=director_name[director_name.find("\n")+1:]
                   director.append(director_name)
                  else:
                   director.append('-')
                  if actor_name.find("Stars")!=-1:
                   actor_name=actor_name[actor_name.find("\n")+1:]
                   stars.append(actor_name)
                  else:
                   stars.append('-')
              else:
                  tmp_name=name[0].strip()
                  if tmp_name.find("Director")!=-1:
                   tmp_name=tmp_name[tmp_name.find("\n")+1:]
                   director.append(name)
                  else:
                   director.append('-')
                  if tmp_name.find("Stars")!=-1:
                   tmp_name=tmp_name[tmp_name.find("\n")+1:]
                   stars.append(tmp_name)
                  else:
                   stars.append('-')
    print(len(titles))
movies_1 = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'video':video,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross,
                       'genres':genres,
                       'genre1':genre1,
                       'genre2':genre2,
                       'genre3':genre3,
                       'plot':plot,
                       'director':director,
                       'stars':stars,
                       'primary_genre':primary_genre
                       })
# Cleaning 'year' column
movies_1['year'] = movies_1['year'].str.extract('(\d+)').astype(str)
movies_1.info()
movies_1=movies_1.replace('-',np.NaN)
movies_1.info()


2000


In [ ]:
movies_1 = movies_1[['movie','genres','director','stars','plot']]

In [ ]:
#droping rows with Null values
movies_1=movies_1.dropna()
movies_1=movies_1.drop_duplicates(subset =['movie'])
movies_1.to_csv('movies.csv')